# dllModel - Python Runner Notebook

![](dllModel.png)

# Python Setup

This is the type setup required before running the model.

For readability the `rtwtypes.h` is converted to [`rtwtypes.py`](https://github.com/dapperfu/python_SimulinkDLL/blob/master/Example1/rtwtypes.py) and in a separate file.

In [1]:
from rtwtypes import *

### ```dllModel.h``` in Python

Using `ctypes.Structure` re-create all of the `dllModel.h` structure.

In [2]:
class B_dllModel_T(ctypes.Structure):
    """Simulink.Parameter SimulinkGlobals"""
    _fields_ = [
        ("SimulationSignal1", real_T),
        ("SignalOut3", real32_T),
        ("SignalOut2", uint16_T),
    ]


class ExtU_dllModel_T(ctypes.Structure):
    """SimulinkGlobal Inputs to Model"""
    _fields_ = [("SignalIin2", uint16_T)]


class ExtY_dllModel_T(ctypes.Structure):
    """SimulinkGlobal Outputs from Model"""
    _fields_ = [("OutputPort2", real32_T)]


class P_dllModel_T(ctypes.Structure):
    """SimulinkGlobal Parameters"""
    _fields_ = [("K2", uint16_T)]


class Timing(ctypes.Structure):
    """ Timing STructure"""
    _fields_ = [
        ("clockTick0", uint32_T),
        ("clockTickH0", uint32_T),
    ]


class tag_RTM_dllModel_T(ctypes.Structure):
    """ tag RTM """
    _fields_ = [
        ("errorStatus", ctypes.c_char_p),
        ("Timing", Timing),
    ]

In [3]:
import os

dll_path = os.path.abspath("dllModel_win64.dll")
dll = ctypes.windll.LoadLibrary(dll_path)

In [4]:
# Block parameters (default storage)
dllModel_P = P_dllModel_T.in_dll(dll, "dllModel_P")
# Block signals (default storage)
dllModel_B = B_dllModel_T.in_dll(dll, "dllModel_B")
# External inputs (root inport signals with default storage)
dllModel_U = ExtU_dllModel_T.in_dll(dll, "dllModel_U")
# External outputs (root outports fed by signals with default storage)
dllModel_Y = ExtY_dllModel_T.in_dll(dll, "dllModel_Y")

In [5]:
"""
 * Exported Global Signals
 *
 * Note: Exported global signals are block signals with an exported global
 * storage class designation.  Code generation will declare the memory for
 * these signals and export their symbols.
 """
SignalIn = real32_T.in_dll(dll, "SignalIn")
SimulationSignal2 = real_T.in_dll(dll, "SimulationSignal2")
SignalOut = real32_T.in_dll(dll, "SignalOut")
"""
 * Exported Global Parameters
 *
 * Note: Exported global parameters are tunable parameters with an exported
 * global storage class designation.  Code generation will declare the memory for
 * these parameters and exports their symbols.
"""
K = real32_T.in_dll(dll, "K")

# Model entry point functions
dllModel_initialize = dll.dllModel_initialize
dllModel_step = dll.dllModel_step
dllModel_terminate = dll.dllModel_terminate

# Real-time Model object
dllModel_M = ctypes.POINTER(tag_RTM_dllModel_T).in_dll(dll, "dllModel_M")

# Running The Model.

Before running the model you will need to run the model init function.

In [6]:
dllModel_initialize();

Take an initial step into the model, function returns the current step number.

In [7]:
dllModel_step()

1

Inspect the simulation time, both in the Block signal structure and in the global variable.

In [8]:
[dllModel_B.SimulationSignal1, SimulationSignal2]

[0.0, c_double(0.0)]

Take another step and inspect the simulation time again.

0.1s matches the discrete step size specified in the model.

![](dllModel_solver.png)


In [9]:
dllModel_step()
[dllModel_B.SimulationSignal1, SimulationSignal2]

[0.1, c_double(0.1)]

### Manipulating Signals

& Reading Outputs

![](dllModel.png)

In [10]:
SignalIn.value = float(2)
SignalOut

c_float(0.0)

In [11]:
dllModel_step()
SignalOut

c_float(2.0)

In [12]:
[dllModel_B.SignalOut2, dllModel_B.SignalOut3, SignalOut]

[0, 2.0, c_float(2.0)]

In [13]:
dllModel_U.SignalIin2 = 1

In [14]:
dllModel_step()
[dllModel_B.SignalOut2, dllModel_B.SignalOut3, SignalOut]

[2, 4.0, c_float(2.0)]

In [15]:
dllModel_M.contents.Timing.clockTick0

4

In [16]:
step = dllModel_step()
assert dllModel_M.contents.Timing.clockTick0 == step